In [1]:
# should already be activated, but just a reminder where ya are :)  
# !conda activate wool_sucking_nn

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats 

%matplotlib inline

In [3]:
# import data
excel_file = pd.read_excel('..\Data\cleaned_and_split_data.xlsx', sheet_name=None)

# Unpack the dataframes into separate variables
X_train = excel_file['X_train']
X_test = excel_file['X_test']
y_train = excel_file['y_train']
y_test = excel_file['y_test']

In [4]:
print(X_train.shape)
X_train.head()

(3965, 7)


,Neuter_status,Breed_group,Aggression_owner,Aggression_cats,Shyness_novel,Shyness_strangers,Grooming
0,1,SIB,1,2,4,4,3
1,1,HCS,1,2,4,4,2
2,0,ORI,1,1,1,1,1
3,1,MCO,1,1,2,2,3
4,1,HCS,1,1,2,1,2


In [5]:
print(X_test.shape)
X_test.head()

(1700, 7)


,Neuter_status,Breed_group,Aggression_owner,Aggression_cats,Shyness_novel,Shyness_strangers,Grooming
0,1,SIB,1,3,3,1,1
1,1,MCO,1,1,2,1,1
2,1,RUS,1,1,4,4,1
3,0,EUR,2,2,2,2,1
4,1,NFO,1,2,3,2,1


In [6]:
print(y_train.shape)
y_train.head()

(3965, 1)


,Wool_sucking_binary
0,0
1,0
2,0
3,0
4,0


In [7]:
print(y_test.shape)
y_test.head()

(1700, 1)


,Wool_sucking_binary
0,1
1,1
2,1
3,0
4,1


## To record model metrics

In [8]:
# dataframe for scores amongst models
master_scores = pd.DataFrame(columns=['Model','Recall', 'F1', 'Precision', 'Accuracy'])

In [9]:
# add_to_master(model_description, y_true, y_predicted, binary=True)
def add_to_master(model_description, y_test, y_pred, binary=True):
    '''
    Adds a new row to the running score DataFrame `master_scores` 
    and functions for adding both binary and multiclass scores. 
    '''
    global master_scores
    if binary == True:
        # Evalution Metrics
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        # add to master_scores
        new_row = {
            'Model': model_description,
            'Recall': round(recall,3), 
            'F1': round(f1, 3), 
            'Precision': round(precision, 3), 
            'Accuracy': round(accuracy, 3)}
        
        master_scores = master_scores.append(new_row, ignore_index=True)   
    
    else:
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='macro') # I chose macro since we have less of the sucklers 
        recall = recall_score(y_test, y_pred, average='macro')       # and I want their accuracy to be meaningful
        f1 = f1_score(y_test, y_pred, average='macro')


        # add to master_scores
        new_row = {
            'Model': model_description,
            'Recall': round(recall,3), 
            'F1': round(f1, 3), 
            'Precision': round(precision, 3), 
            'Accuracy': round(accuracy, 3)}
        master_scores = master_scores.append(new_row, ignore_index=True)
    return master_scores

# Prep for tensorflow.keras

In [10]:
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
# for reproducibility 
tf.random.set_seed(42)
np.random.seed(42)

In [11]:
# one-hot breed 
X_train_encoded = pd.get_dummies(X_train, columns=['Breed_group'])
X_test_encoded = pd.get_dummies(X_test, columns=['Breed_group'])

In [12]:
X_train_encoded

,Neuter_status,Aggression_owner,Aggression_cats,Shyness_novel,Shyness_strangers,Grooming,Breed_group_ABY,Breed_group_BEN,Breed_group_BRI,Breed_group_BUR,...,Breed_group_MCO,Breed_group_NFO,Breed_group_ORI,Breed_group_PER,Breed_group_RAG,Breed_group_RUS,Breed_group_SBI,Breed_group_SIB,Breed_group_TUV,Breed_group_other
0,1,1,2,4,4,3,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1,1,2,4,4,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,1,1,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,1,1,1,2,2,3,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,1,1,1,2,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3960,1,1,3,1,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3961,1,2,2,2,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3962,1,1,1,3,4,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3963,1,1,1,4,4,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Plain Jane

In [13]:
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(len(X_train_encoded.columns),)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [14]:
def recall(y_true, y_pred):
    true_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
    possible_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + tf.keras.backend.epsilon())
    return recall

tf.keras.utils.get_custom_objects()['recall'] = recall

In [15]:
model.compile(optimizer='adam', 
             loss='binary_crossentropy',
             metrics=['accuracy', 'recall'])

In [16]:
model.fit(X_train_encoded, y_train, epochs=10, batch_size=32)

Epoch 1/10
124/124 [==============================] - 1s 900us/step - loss: 0.6245 - accuracy: 0.6835 - recall: 0.0334
Epoch 2/10
124/124 [==============================] - 0s 900us/step - loss: 0.6061 - accuracy: 0.6827 - recall: 0.1261
Epoch 3/10
124/124 [==============================] - 0s 892us/step - loss: 0.5977 - accuracy: 0.6885 - recall: 0.1762
Epoch 4/10
124/124 [==============================] - 0s 884us/step - loss: 0.5927 - accuracy: 0.6916 - recall: 0.1534
Epoch 5/10
124/124 [==============================] - 0s 900us/step - loss: 0.5880 - accuracy: 0.6913 - recall: 0.1845
Epoch 6/10
124/124 [==============================] - 0s 916us/step - loss: 0.5868 - accuracy: 0.6926 - recall: 0.2057
Epoch 7/10
124/124 [==============================] - 0s 924us/step - loss: 0.5858 - accuracy: 0.6981 - recall: 0.2210
Epoch 8/10
124/124 [==============================] - 0s 892us/step - loss: 0.5798 - accuracy: 0.6971 - recall: 0.2013
Epoch 9/10
124/124 [============================

In [17]:
test_loss, test_acc, test_recall = model.evaluate(X_test_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

54/54 [==============================] - 0s 734us/step - loss: 0.5863 - accuracy: 0.6924 - recall: 0.2179
Test Accuracy: 0.6923529505729675
Test Loss: 0.5862632989883423
Test Recall: 0.21787841618061066


In [18]:
y_pred = model.predict(X_test_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
# add_to_master(model_description, y_true, y_predicted, binary=True)
add_to_master("1 hidden, 64 nodes per, adam", y_test, y_pred)
cm

54/54 [==============================] - 0s 621us/step


C:\Users\gwent\AppData\Local\Temp\ipykernel_4284\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[1055,  104],
       [ 419,  122]], dtype=int64)

In [19]:
master_scores

,Model,Recall,F1,Precision,Accuracy
0,"1 hidden, 64 nodes per, adam",0.226,0.318,0.54,0.692


Not doing too hot, but also seems to be doing better than initial models in Cat-pstone 1. 

## Adagrad optimizer

In [20]:
model2 = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(len(X_train_encoded.columns),)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [21]:
# Adagrad better for when feataures vary in magnitude and frequency 
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.01)
model2.compile(optimizer=optimizer, 
             loss='binary_crossentropy',
             metrics=['accuracy', 'recall'])

In [22]:
model2.fit(X_train_encoded, y_train, epochs=10, batch_size=32)

Epoch 1/10
124/124 [==============================] - 0s 811us/step - loss: 0.6439 - accuracy: 0.6653 - recall: 0.0595
Epoch 2/10
124/124 [==============================] - 0s 819us/step - loss: 0.6229 - accuracy: 0.6820 - recall: 0.0028
Epoch 3/10
124/124 [==============================] - 0s 819us/step - loss: 0.6149 - accuracy: 0.6810 - recall: 0.0351
Epoch 4/10
124/124 [==============================] - 0s 843us/step - loss: 0.6098 - accuracy: 0.6845 - recall: 0.0583
Epoch 5/10
124/124 [==============================] - 0s 835us/step - loss: 0.6058 - accuracy: 0.6883 - recall: 0.0919
Epoch 6/10
124/124 [==============================] - 0s 835us/step - loss: 0.6021 - accuracy: 0.6888 - recall: 0.1207
Epoch 7/10
124/124 [==============================] - 0s 819us/step - loss: 0.5997 - accuracy: 0.6910 - recall: 0.1250
Epoch 8/10
124/124 [==============================] - 0s 835us/step - loss: 0.5980 - accuracy: 0.6905 - recall: 0.1295
Epoch 9/10
124/124 [============================

In [23]:
test_loss, test_acc, test_recall = model2.evaluate(X_test_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

54/54 [==============================] - 0s 715us/step - loss: 0.5994 - accuracy: 0.6818 - recall: 0.1569
Test Accuracy: 0.6817647218704224
Test Loss: 0.5994249582290649
Test Recall: 0.15686042606830597


In [24]:
y_pred = model2.predict(X_test_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("1 hidden, 64 nodes per, adagrad", y_test, y_pred)
cm

54/54 [==============================] - 0s 602us/step


C:\Users\gwent\AppData\Local\Temp\ipykernel_4284\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[1073,   86],
       [ 455,   86]], dtype=int64)

## Scaled with Plain Jane and Adagrad

In [25]:
scaler = StandardScaler()
scaler.fit(X_train_encoded)

StandardScaler()

In [26]:
X_train_scaled_encoded = scaler.transform(X_train_encoded)
X_test_scaled_encoded = scaler.transform(X_test_encoded)

In [27]:
X_train_scaled_encoded.shape

(3965, 25)

In [28]:
# Plain Jane + scaled
model3 = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(25,)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model3.compile(optimizer='adam', 
             loss='binary_crossentropy',
             metrics=['accuracy', 'recall'])

model3.fit(X_train_scaled_encoded, y_train, epochs=10, batch_size=32)

test_loss, test_acc, test_recall = model3.evaluate(X_test_scaled_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

y_pred = model3.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("1 hidden, 64 nodes per, adam, scaled", y_test, y_pred)
cm

Epoch 1/10
124/124 [==============================] - 0s 851us/step - loss: 0.6132 - accuracy: 0.6739 - recall: 0.0788
Epoch 2/10
124/124 [==============================] - 0s 892us/step - loss: 0.5877 - accuracy: 0.6910 - recall: 0.1642
Epoch 3/10
124/124 [==============================] - 0s 876us/step - loss: 0.5820 - accuracy: 0.6926 - recall: 0.2217
Epoch 4/10
124/124 [==============================] - 0s 892us/step - loss: 0.5784 - accuracy: 0.6966 - recall: 0.1850
Epoch 5/10
124/124 [==============================] - 0s 884us/step - loss: 0.5735 - accuracy: 0.7032 - recall: 0.2211
Epoch 6/10
124/124 [==============================] - 0s 892us/step - loss: 0.5727 - accuracy: 0.7037 - recall: 0.2345
Epoch 7/10
124/124 [==============================] - 0s 892us/step - loss: 0.5710 - accuracy: 0.7019 - recall: 0.2441
Epoch 8/10
124/124 [==============================] - 0s 876us/step - loss: 0.5678 - accuracy: 0.7087 - recall: 0.2554
Epoch 9/10
124/124 [============================

C:\Users\gwent\AppData\Local\Temp\ipykernel_4284\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[984, 175],
       [391, 150]], dtype=int64)

In [29]:
# adagrad + scaled 
model4 = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(25,)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# Adagrad better for when feataures vary in magnitude and frequency 
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.01)
model4.compile(optimizer=optimizer, 
             loss='binary_crossentropy',
             metrics=['accuracy', 'recall'])

model4.fit(X_train_scaled_encoded, y_train, epochs=10, batch_size=32)

test_loss, test_acc, test_recall = model4.evaluate(X_test_scaled_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

y_pred = model4.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("1 hidden, 64 nodes per, adagrad, scaled", y_test, y_pred)
cm

Epoch 1/10
124/124 [==============================] - 0s 835us/step - loss: 0.6366 - accuracy: 0.6502 - recall: 0.0893
Epoch 2/10
124/124 [==============================] - 0s 851us/step - loss: 0.6025 - accuracy: 0.6817 - recall: 0.0365
Epoch 3/10
124/124 [==============================] - 0s 835us/step - loss: 0.5942 - accuracy: 0.6890 - recall: 0.1029
Epoch 4/10
124/124 [==============================] - 0s 868us/step - loss: 0.5898 - accuracy: 0.6913 - recall: 0.1275
Epoch 5/10
124/124 [==============================] - 0s 933us/step - loss: 0.5871 - accuracy: 0.6979 - recall: 0.1513
Epoch 6/10
124/124 [==============================] - 0s 949us/step - loss: 0.5850 - accuracy: 0.7009 - recall: 0.1813
Epoch 7/10
124/124 [==============================] - 0s 989us/step - loss: 0.5832 - accuracy: 0.6971 - recall: 0.1735
Epoch 8/10
124/124 [==============================] - 0s 949us/step - loss: 0.5820 - accuracy: 0.7011 - recall: 0.2003
Epoch 9/10
124/124 [============================

C:\Users\gwent\AppData\Local\Temp\ipykernel_4284\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[1056,  103],
       [ 444,   97]], dtype=int64)

In [30]:
master_scores.sort_values('Recall', ascending=False)

,Model,Recall,F1,Precision,Accuracy
2,"1 hidden, 64 nodes per, adam, scaled",0.277,0.346,0.462,0.667
0,"1 hidden, 64 nodes per, adam",0.226,0.318,0.540,0.692
3,"1 hidden, 64 nodes per, adagrad, scaled",0.179,0.262,0.485,0.678
1,"1 hidden, 64 nodes per, adagrad",0.159,0.241,0.500,0.682


In [31]:
def create_mlp(nodes, optimizer):
    # Define the model architecture
    model = keras.Sequential([
    keras.layers.Dense(nodes, activation='relu', input_shape=(25,)),
    keras.layers.Dense(nodes, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

    model.compile(optimizer=optimizer, 
             loss='binary_crossentropy',
             metrics=['accuracy', 'recall'])
    
    return model

In [32]:
model5 = create_mlp(100, 'adam')

In [33]:
model5.fit(X_train_scaled_encoded, y_train, epochs=10, batch_size=32)

test_loss, test_acc, test_recall = model5.evaluate(X_test_scaled_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

y_pred = model5.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("1 hidden, 100 nodes, adam", y_test, y_pred)
cm

Epoch 1/10
124/124 [==============================] - 0s 965us/step - loss: 0.6079 - accuracy: 0.6797 - recall: 0.1172
Epoch 2/10
124/124 [==============================] - 0s 973us/step - loss: 0.5879 - accuracy: 0.6966 - recall: 0.1844
Epoch 3/10
124/124 [==============================] - 0s 973us/step - loss: 0.5817 - accuracy: 0.6979 - recall: 0.2260
Epoch 4/10
124/124 [==============================] - 0s 973us/step - loss: 0.5786 - accuracy: 0.7014 - recall: 0.1979
Epoch 5/10
124/124 [==============================] - 0s 981us/step - loss: 0.5715 - accuracy: 0.7064 - recall: 0.2255
Epoch 6/10
124/124 [==============================] - 0s 981us/step - loss: 0.5722 - accuracy: 0.7069 - recall: 0.2462
Epoch 7/10
124/124 [==============================] - 0s 973us/step - loss: 0.5705 - accuracy: 0.7079 - recall: 0.2605
Epoch 8/10
124/124 [==============================] - 0s 973us/step - loss: 0.5659 - accuracy: 0.7110 - recall: 0.2628
Epoch 9/10
124/124 [============================

C:\Users\gwent\AppData\Local\Temp\ipykernel_4284\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[998, 161],
       [396, 145]], dtype=int64)

In [34]:
model6 = create_mlp(32, 'adam')
model6.fit(X_train_scaled_encoded, y_train, epochs=10, batch_size=32)

test_loss, test_acc, test_recall = model6.evaluate(X_test_scaled_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

y_pred = model6.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("1 hidden, 32 nodes, adam", y_test, y_pred)
cm

Epoch 1/10
124/124 [==============================] - 0s 835us/step - loss: 0.6174 - accuracy: 0.6792 - recall: 0.0502
Epoch 2/10
124/124 [==============================] - 0s 876us/step - loss: 0.5920 - accuracy: 0.6941 - recall: 0.1011
Epoch 3/10
124/124 [==============================] - 0s 851us/step - loss: 0.5863 - accuracy: 0.6981 - recall: 0.1709
Epoch 4/10
124/124 [==============================] - 0s 851us/step - loss: 0.5822 - accuracy: 0.6976 - recall: 0.1704
Epoch 5/10
124/124 [==============================] - 0s 843us/step - loss: 0.5781 - accuracy: 0.7057 - recall: 0.1878
Epoch 6/10
124/124 [==============================] - 0s 900us/step - loss: 0.5771 - accuracy: 0.7029 - recall: 0.2089
Epoch 7/10
124/124 [==============================] - 0s 868us/step - loss: 0.5751 - accuracy: 0.7047 - recall: 0.2079
Epoch 8/10
124/124 [==============================] - 0s 851us/step - loss: 0.5735 - accuracy: 0.7102 - recall: 0.2380
Epoch 9/10
124/124 [============================

C:\Users\gwent\AppData\Local\Temp\ipykernel_4284\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[1026,  133],
       [ 413,  128]], dtype=int64)

In [35]:
model7 = create_mlp(250, 'adam')
model7.fit(X_train_scaled_encoded, y_train, epochs=10, batch_size=32)

test_loss, test_acc, test_recall = model7.evaluate(X_test_scaled_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

y_pred = model7.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("1 hidden, 250 nodes, adam", y_test, y_pred)
cm

Epoch 1/10
124/124 [==============================] - 0s 1ms/step - loss: 0.6079 - accuracy: 0.6820 - recall: 0.1615
Epoch 2/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5876 - accuracy: 0.6951 - recall: 0.2009
Epoch 3/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5800 - accuracy: 0.6984 - recall: 0.2258
Epoch 4/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5769 - accuracy: 0.7006 - recall: 0.1998
Epoch 5/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5683 - accuracy: 0.7122 - recall: 0.2491
Epoch 6/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5689 - accuracy: 0.7102 - recall: 0.2545
Epoch 7/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5669 - accuracy: 0.7084 - recall: 0.2736
Epoch 8/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5609 - accuracy: 0.7135 - recall: 0.2744
Epoch 9/10
124/124 [==============================] - 0s 1ms/ste

C:\Users\gwent\AppData\Local\Temp\ipykernel_4284\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[982, 177],
       [395, 146]], dtype=int64)

In [36]:
# definitely overfits
model8 = create_mlp(300, 'adam')
model8.fit(X_train_scaled_encoded, y_train, epochs=10, batch_size=32)

test_loss, test_acc, test_recall = model8.evaluate(X_test_scaled_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

y_pred = model8.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("1 hidden, 300 nodes, adam", y_test, y_pred)
cm

Epoch 1/10
124/124 [==============================] - 1s 1ms/step - loss: 0.6076 - accuracy: 0.6850 - recall: 0.1517
Epoch 2/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5872 - accuracy: 0.6916 - recall: 0.2058
Epoch 3/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5797 - accuracy: 0.6979 - recall: 0.2225
Epoch 4/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5770 - accuracy: 0.6999 - recall: 0.2055
Epoch 5/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5691 - accuracy: 0.7074 - recall: 0.2443
Epoch 6/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5713 - accuracy: 0.7064 - recall: 0.2649
Epoch 7/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5679 - accuracy: 0.7069 - recall: 0.2698
Epoch 8/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5610 - accuracy: 0.7122 - recall: 0.2732
Epoch 9/10
124/124 [==============================] - 0s 1ms/ste

C:\Users\gwent\AppData\Local\Temp\ipykernel_4284\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[986, 173],
       [391, 150]], dtype=int64)

In [37]:
master_scores.sort_values('Recall', ascending=False)

,Model,Recall,F1,Precision,Accuracy
2,"1 hidden, 64 nodes per, adam, scaled",0.277,0.346,0.462,0.667
7,"1 hidden, 300 nodes, adam",0.277,0.347,0.464,0.668
6,"1 hidden, 250 nodes, adam",0.270,0.338,0.452,0.664
4,"1 hidden, 100 nodes, adam",0.268,0.342,0.474,0.672
5,"1 hidden, 32 nodes, adam",0.237,0.319,0.490,0.679
0,"1 hidden, 64 nodes per, adam",0.226,0.318,0.540,0.692
3,"1 hidden, 64 nodes per, adagrad, scaled",0.179,0.262,0.485,0.678
1,"1 hidden, 64 nodes per, adagrad",0.159,0.241,0.500,0.682


Okay so we are pretty similar in performance to the logistic regression models from cat-pstone 1 on every metric except recall (and the part it plays in F1, of course). Let's see if over or undersampling with the top performing model architecture (model1). 

In [38]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

In [39]:
rus = RandomUnderSampler()
X_resampled_rus, y_resampled_rus = rus.fit_resample(X_train_scaled_encoded, y_train)

ros = RandomOverSampler()
X_resampled_ros, y_resampled_ros = ros.fit_resample(X_train_scaled_encoded, y_train)

In [40]:
# RUS 
model9 = create_mlp(64, 'adam')
model9.fit(X_resampled_rus, y_resampled_rus, epochs=10, batch_size=32)

y_pred = model9.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("RUS 1 hidden, 64 nodes, adam", y_test, y_pred)
cm

Epoch 1/10
79/79 [==============================] - 0s 870us/step - loss: 0.6755 - accuracy: 0.5884 - recall: 0.5741
Epoch 2/10
79/79 [==============================] - 0s 895us/step - loss: 0.6502 - accuracy: 0.6288 - recall: 0.6164
Epoch 3/10
79/79 [==============================] - 0s 921us/step - loss: 0.6430 - accuracy: 0.6387 - recall: 0.6308
Epoch 4/10
79/79 [==============================] - 0s 895us/step - loss: 0.6375 - accuracy: 0.6375 - recall: 0.6230
Epoch 5/10
79/79 [==============================] - 0s 908us/step - loss: 0.6334 - accuracy: 0.6462 - recall: 0.6276
Epoch 6/10
79/79 [==============================] - 0s 895us/step - loss: 0.6298 - accuracy: 0.6502 - recall: 0.6600
Epoch 7/10
79/79 [==============================] - 0s 908us/step - loss: 0.6245 - accuracy: 0.6585 - recall: 0.6378
Epoch 8/10
79/79 [==============================] - 0s 997us/step - loss: 0.6206 - accuracy: 0.6668 - recall: 0.6622
Epoch 9/10
79/79 [==============================] - 0s 946us/ste

C:\Users\gwent\AppData\Local\Temp\ipykernel_4284\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[732, 427],
       [223, 318]], dtype=int64)

In [41]:
# ROS 
model10 = create_mlp(64, 'adam')
model10.fit(X_resampled_ros, y_resampled_ros, epochs=10, batch_size=32)

y_pred = model10.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("ROS 1 hidden, 64 nodes, adam", y_test, y_pred)
cm

Epoch 1/10
169/169 [==============================] - 1s 861us/step - loss: 0.6704 - accuracy: 0.5877 - recall: 0.5971
Epoch 2/10
169/169 [==============================] - 0s 879us/step - loss: 0.6443 - accuracy: 0.6287 - recall: 0.6066
Epoch 3/10
169/169 [==============================] - 0s 879us/step - loss: 0.6360 - accuracy: 0.6417 - recall: 0.6290
Epoch 4/10
169/169 [==============================] - 0s 891us/step - loss: 0.6290 - accuracy: 0.6461 - recall: 0.6291
Epoch 5/10
169/169 [==============================] - 0s 879us/step - loss: 0.6209 - accuracy: 0.6535 - recall: 0.6364
Epoch 6/10
169/169 [==============================] - 0s 879us/step - loss: 0.6175 - accuracy: 0.6583 - recall: 0.6634
Epoch 7/10
169/169 [==============================] - 0s 879us/step - loss: 0.6138 - accuracy: 0.6552 - recall: 0.6483
Epoch 8/10
169/169 [==============================] - 0s 873us/step - loss: 0.6096 - accuracy: 0.6624 - recall: 0.6428
Epoch 9/10
169/169 [============================

C:\Users\gwent\AppData\Local\Temp\ipykernel_4284\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[836, 323],
       [291, 250]], dtype=int64)

In [42]:
master_scores.sort_values('Recall', ascending=False)

,Model,Recall,F1,Precision,Accuracy
8,"RUS 1 hidden, 64 nodes, adam",0.588,0.495,0.427,0.618
9,"ROS 1 hidden, 64 nodes, adam",0.462,0.449,0.436,0.639
2,"1 hidden, 64 nodes per, adam, scaled",0.277,0.346,0.462,0.667
7,"1 hidden, 300 nodes, adam",0.277,0.347,0.464,0.668
6,"1 hidden, 250 nodes, adam",0.270,0.338,0.452,0.664
4,"1 hidden, 100 nodes, adam",0.268,0.342,0.474,0.672
5,"1 hidden, 32 nodes, adam",0.237,0.319,0.490,0.679
0,"1 hidden, 64 nodes per, adam",0.226,0.318,0.540,0.692
3,"1 hidden, 64 nodes per, adagrad, scaled",0.179,0.262,0.485,0.678
1,"1 hidden, 64 nodes per, adagrad",0.159,0.241,0.500,0.682


In [47]:
# RUS 
model11 = create_mlp(64, 'adam')
model11.fit(X_resampled_rus, y_resampled_rus, epochs=10, batch_size=25)

y_pred = model11.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("RUS 1 hidden, 64 nodes, adam, batch 25", y_test, y_pred)
cm

Epoch 1/10
101/101 [==============================] - 0s 928us/step - loss: 0.6834 - accuracy: 0.5697 - recall: 0.5898
Epoch 2/10
101/101 [==============================] - 0s 964us/step - loss: 0.6529 - accuracy: 0.6169 - recall: 0.6031
Epoch 3/10
101/101 [==============================] - 0s 1ms/step - loss: 0.6457 - accuracy: 0.6280 - recall: 0.6251
Epoch 4/10
101/101 [==============================] - 0s 977us/step - loss: 0.6393 - accuracy: 0.6335 - recall: 0.6208
Epoch 5/10
101/101 [==============================] - 0s 958us/step - loss: 0.6355 - accuracy: 0.6462 - recall: 0.6295
Epoch 6/10
101/101 [==============================] - 0s 908us/step - loss: 0.6286 - accuracy: 0.6494 - recall: 0.6607
Epoch 7/10
101/101 [==============================] - 0s 968us/step - loss: 0.6266 - accuracy: 0.6533 - recall: 0.6427
Epoch 8/10
101/101 [==============================] - 0s 928us/step - loss: 0.6226 - accuracy: 0.6605 - recall: 0.6630
Epoch 9/10
101/101 [==============================

C:\Users\gwent\AppData\Local\Temp\ipykernel_4284\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[736, 423],
       [231, 310]], dtype=int64)

In [48]:
master_scores.sort_values('Recall', ascending=False).head()

,Model,Recall,F1,Precision,Accuracy
8,"RUS 1 hidden, 64 nodes, adam",0.588,0.495,0.427,0.618
12,"RUS 1 hidden, 64 nodes, adam, batch 25",0.573,0.487,0.423,0.615
10,"RUS 1 hidden, 100 nodes, adam",0.527,0.465,0.417,0.615
11,"RUS 1 hidden, 64 nodes, adam",0.518,0.471,0.432,0.630
9,"ROS 1 hidden, 64 nodes, adam",0.462,0.449,0.436,0.639
